In [2]:
import pandas as pd
import numpy as np
import time
from lifelines.utils import concordance_index
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
from sksurv.ensemble import RandomSurvivalForest
from sklearn.metrics import make_scorer

In [3]:
df_train = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

In [4]:
# select features
X_surv_train = df_train.drop(columns=['TIMETOEVENT', 'MORTSTAT'])
X_surv_test = df_test.drop(columns=['TIMETOEVENT', 'MORTSTAT'])
# select target
y_surv_train = np.array([(event, time) for event, time in zip(df_train['MORTSTAT'], df_train['TIMETOEVENT'])], dtype=[('MORTSTAT', bool), ('TIMETOEVENT', float)])
y_surv_test = np.array([(event, time) for event, time in zip(df_test['MORTSTAT'], df_test['TIMETOEVENT'])], dtype=[('MORTSTAT', bool), ('TIMETOEVENT', float)])

In [5]:
def c_index_classification(y_true, y_pred_proba, times):
    y_true = np.asarray(y_true)
    y_pred_proba = np.asarray(y_pred_proba)
    times = np.asarray(times)
    cc = dc = tp = 0

    n = len(y_true)
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if times[i] < times[j] and y_true[i] == 1:
                if y_pred_proba[i] < y_pred_proba[j]:
                    cc += 1
                elif y_pred_proba[i] > y_pred_proba[j]:
                    dc += 1
                else:
                    tp += 1

        
    return (cc + 0.5 * tp) / (cc + dc + tp)

In [6]:
# define scorer

In [7]:
def ll3y(estimator, X, y):
    event_indicators = np.where((y["MORTSTAT"] == 1) & (y["TIMETOEVENT"] <= 3), 0, 1)

    # get predicted survival functions
    surv_funcs = estimator.predict_survival_function(X)

    # get time points (assuming all survival functions have same time grid)
    time_points = surv_funcs[0].x

    # extract survival probabilities at 3 years
    surv_prob_3y = np.array([fn(time_points[3]) for fn in surv_funcs])

    # compute log-loss
    logloss = log_loss(event_indicators, surv_prob_3y)

    return -logloss  

In [8]:
def bs3y(estimator, X, y):
    event_indicators = np.where((y["MORTSTAT"] == 1) & (y["TIMETOEVENT"] <= 3), 0, 1)

    # get predicted survival functions
    surv_funcs = estimator.predict_survival_function(X)

    # get time points (assuming all survival functions have same time grid)
    time_points = surv_funcs[0].x

    # extract survival probabilities at 3 years
    surv_prob_3y = np.array([fn(time_points[3]) for fn in surv_funcs])

    # compute log-loss
    bs = brier_score_loss(event_indicators, surv_prob_3y)

    return -bs  

In [9]:
# RandomSurvivalForest

In [10]:
#______________________________________________________________________________________________________________________________

In [11]:
# hyperparamter search (scorer=log-loss)

In [12]:
param_grid_rst = {'n_estimators': [100, 150, 200],
                  'max_samples': [2000, 3000, 4000],
                  'max_leaf_nodes': [20, 30, 40, 50],
                  'max_features': [20, 30, 40, None]}

grid_search_rst1 = GridSearchCV(estimator=RandomSurvivalForest(random_state=42), param_grid=param_grid_rst, scoring=ll3y, cv=4, n_jobs=-1, verbose=2)

grid_search_rst1.fit(X_surv_train, y_surv_train)
print(grid_search_rst1.best_params_)

Fitting 4 folds for each of 144 candidates, totalling 576 fits


C:\ProgramData\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'max_features': 30, 'max_leaf_nodes': 50, 'max_samples': 4000, 'n_estimators': 200}


In [13]:
results_df = pd.DataFrame(grid_search_rst1.cv_results_)

# extract relevant columns
results_df = results_df[['rank_test_score', 'param_n_estimators', 'param_max_samples', 'param_max_leaf_nodes', 'param_max_features', 'mean_test_score']]

# convert negative log-loss to positive (since scoring='neg_log_loss')
results_df['ll3y'] = (- results_df['mean_test_score']).round(4)
results_df = results_df.drop(columns=['mean_test_score'])

results_df = results_df.sort_values(by='rank_test_score')
results_df.head(3)

,rank_test_score,param_n_estimators,param_max_samples,param_max_leaf_nodes,param_max_features,ll3y
71,1,200,4000,50,30,0.1151
68,2,200,3000,50,30,0.1152
69,3,100,4000,50,30,0.1152


In [14]:
# RSF1L

In [15]:
rsf1l = RandomSurvivalForest(random_state=42, 
                             n_estimators=200, max_leaf_nodes=50, max_features=30, max_samples=4000)

start_time = time.time()
rsf1l.fit(X_surv_train, y_surv_train)
end_time = time.time()
print(f"Time for one model: {end_time - start_time:.2f} seconds")

Time for one model: 16.20 seconds


In [16]:
# predict survival function
surv_probs_train = rsf1l.predict_survival_function(X_surv_train)
surv_probs_test = rsf1l.predict_survival_function(X_surv_test)

# all time-to-event points
time_points_train = surv_probs_train[0].x
time_points_test = surv_probs_test[0].x 
 
# convert survival probabilities to df
df_surv_probs_train = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_train],  
    columns=time_points_train)
df_surv_probs_test = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_test],  
    columns=time_points_train)

# surv probabilites after 1 and 3 years
suvr_prob_3y_train = df_surv_probs_train.iloc[:,3]
suvr_prob_3y_test = df_surv_probs_test.iloc[:,3]

# extract status
# get 3-year event status based on TIMETOEVENT and MORTSTAT
status_3y_train_surv = np.where((df_train["MORTSTAT"] == 1) & (df_train["TIMETOEVENT"] <= 3), 0, 1)
status_3y_test_surv = np.where((df_test["MORTSTAT"] == 1) & (df_test["TIMETOEVENT"] <= 3), 0, 1)

# log-loss
log_loss_3y_train = log_loss(status_3y_train_surv, suvr_prob_3y_train)
log_loss_3y_test = log_loss(status_3y_test_surv, suvr_prob_3y_test)

# compute Brier
bs_train_3y = brier_score_loss(status_3y_train_surv, suvr_prob_3y_train)
bs_test_3y = brier_score_loss(status_3y_test_surv, suvr_prob_3y_test)

# c-index
c_is = c_index_classification(df_train["MORTSTAT"], suvr_prob_3y_train, df_train['TIMETOEVENT'])

print(f'Log Loss is: {log_loss_3y_train:.4f}')
print(f'Log Loss os: {log_loss_3y_test:.4f}')
print(f'BS is: {bs_train_3y:.4f}')
print(f'BS os: {bs_test_3y:.4f}')
print(f'C is: {c_is:.4f}')

Log Loss is: 0.1115
Log Loss os: 0.1117
BS is: 0.0298
BS os: 0.0296
C is: 0.8925


In [17]:
# RSF2L

In [18]:
rsf2l = RandomSurvivalForest(random_state=42, 
                             n_estimators=200, max_leaf_nodes=50, max_features=30, max_samples=3000)

start_time = time.time()
rsf2l.fit(X_surv_train, y_surv_train)
end_time = time.time()
print(f"Time for one model: {end_time - start_time:.2f} seconds")

Time for one model: 9.69 seconds


In [19]:
# predict survival function
surv_probs_train = rsf2l.predict_survival_function(X_surv_train)
surv_probs_test = rsf2l.predict_survival_function(X_surv_test)

# all time-to-event points
time_points_train = surv_probs_train[0].x
time_points_test = surv_probs_test[0].x 
 
# convert survival probabilities to df
df_surv_probs_train = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_train],  
    columns=time_points_train)
df_surv_probs_test = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_test],  
    columns=time_points_train)

# surv probabilites after 1 and 3 years
suvr_prob_3y_train = df_surv_probs_train.iloc[:,3]
suvr_prob_3y_test = df_surv_probs_test.iloc[:,3]

# extract status
# get 3-year event status based on TIMETOEVENT and MORTSTAT
status_3y_train_surv = np.where((df_train["MORTSTAT"] == 1) & (df_train["TIMETOEVENT"] <= 3), 0, 1)
status_3y_test_surv = np.where((df_test["MORTSTAT"] == 1) & (df_test["TIMETOEVENT"] <= 3), 0, 1)

# log-loss
log_loss_3y_train = log_loss(status_3y_train_surv, suvr_prob_3y_train)
log_loss_3y_test = log_loss(status_3y_test_surv, suvr_prob_3y_test)

# compute Brier
bs_train_3y = brier_score_loss(status_3y_train_surv, suvr_prob_3y_train)
bs_test_3y = brier_score_loss(status_3y_test_surv, suvr_prob_3y_test)

# c-index
c_is = c_index_classification(df_train["MORTSTAT"], suvr_prob_3y_train, df_train['TIMETOEVENT'])

print(f'Log Loss is: {log_loss_3y_train:.4f}')
print(f'Log Loss os: {log_loss_3y_test:.4f}')
print(f'BS is: {bs_train_3y:.4f}')
print(f'BS os: {bs_test_3y:.4f}')
print(f'C is: {c_is:.4f}')

Log Loss is: 0.1115
Log Loss os: 0.1124
BS is: 0.0297
BS os: 0.0296
C is: 0.8920


In [20]:
# RSF3L

In [21]:
rsf3l = RandomSurvivalForest(random_state=42,
                             n_estimators=100, max_leaf_nodes=50, max_features=30, max_samples=4000)

start_time = time.time()
rsf3l.fit(X_surv_train, y_surv_train)
end_time = time.time()
print(f"Time for one model: {end_time - start_time:.2f} seconds")

Time for one model: 6.40 seconds


In [22]:
# predict survival function
surv_probs_train = rsf3l.predict_survival_function(X_surv_train)
surv_probs_test = rsf3l.predict_survival_function(X_surv_test)

# all time-to-event points
time_points_train = surv_probs_train[0].x
time_points_test = surv_probs_test[0].x 
 
# convert survival probabilities to df
df_surv_probs_train = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_train],  
    columns=time_points_train)
df_surv_probs_test = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_test],  
    columns=time_points_train)

# surv probabilites after 1 and 3 years
suvr_prob_3y_train = df_surv_probs_train.iloc[:,3]
suvr_prob_3y_test = df_surv_probs_test.iloc[:,3]

# extract status
# get 3-year event status based on TIMETOEVENT and MORTSTAT
status_3y_train_surv = np.where((df_train["MORTSTAT"] == 1) & (df_train["TIMETOEVENT"] <= 3), 0, 1)
status_3y_test_surv = np.where((df_test["MORTSTAT"] == 1) & (df_test["TIMETOEVENT"] <= 3), 0, 1)

# log-loss
log_loss_3y_train = log_loss(status_3y_train_surv, suvr_prob_3y_train)
log_loss_3y_test = log_loss(status_3y_test_surv, suvr_prob_3y_test)

# compute Brier
bs_train_3y = brier_score_loss(status_3y_train_surv, suvr_prob_3y_train)
bs_test_3y = brier_score_loss(status_3y_test_surv, suvr_prob_3y_test)

# c-index
c_is = c_index_classification(df_train["MORTSTAT"], suvr_prob_3y_train, df_train['TIMETOEVENT'])

print(f'Log Loss is: {log_loss_3y_train:.4f}')
print(f'Log Loss os: {log_loss_3y_test:.4f}')
print(f'BS is: {bs_train_3y:.4f}')
print(f'BS os: {bs_test_3y:.4f}')
print(f'C is: {c_is:.4f}')

Log Loss is: 0.1117
Log Loss os: 0.1117
BS is: 0.0298
BS os: 0.0296
C is: 0.8917


In [23]:
#______________________________________________________________________________________________________________________________

In [24]:
# hyperparamter search (scorer=Brier-score)

In [25]:
grid_search_rst2 = GridSearchCV(estimator=RandomSurvivalForest(random_state=42), param_grid=param_grid_rst, scoring=bs3y, cv=4, n_jobs=-1, verbose=2)

grid_search_rst2.fit(X_surv_train, y_surv_train)
print(grid_search_rst2.best_params_)

Fitting 4 folds for each of 144 candidates, totalling 576 fits
{'max_features': 40, 'max_leaf_nodes': 50, 'max_samples': 2000, 'n_estimators': 200}


In [26]:
results_df = pd.DataFrame(grid_search_rst2.cv_results_)

# extract relevant columns
results_df = results_df[['rank_test_score', 'param_n_estimators', 'param_max_samples', 'param_max_leaf_nodes', 'param_max_features', 'mean_test_score']]

# convert negative log-loss to positive (since scoring='neg_log_loss')
results_df['bs3y'] = (- results_df['mean_test_score']).round(4)
results_df = results_df.drop(columns=['mean_test_score'])

results_df = results_df.sort_values(by='rank_test_score')
results_df.head(3)

,rank_test_score,param_n_estimators,param_max_samples,param_max_leaf_nodes,param_max_features,bs3y
101,1,200,2000,50,40,0.0302
100,2,150,2000,50,40,0.0302
137,3,200,2000,50,None,0.0302


In [27]:
# RSF1B

In [28]:
rsf1b = RandomSurvivalForest(random_state=42,
                             n_estimators=200, max_leaf_nodes=50, max_features=40, max_samples=2000)
start_time = time.time()
rsf1b.fit(X_surv_train, y_surv_train)
end_time = time.time()
print(f"Time for one model: {end_time - start_time:.2f} seconds")

Time for one model: 7.62 seconds


In [29]:
# predict survival function
surv_probs_train = rsf1b.predict_survival_function(X_surv_train)
surv_probs_test = rsf1b.predict_survival_function(X_surv_test)

# all time-to-event points
time_points_train = surv_probs_train[0].x
time_points_test = surv_probs_test[0].x 
 
# convert survival probabilities to df
df_surv_probs_train = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_train],  
    columns=time_points_train)
df_surv_probs_test = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_test],  
    columns=time_points_train)

# surv probabilites after 1 and 3 years
suvr_prob_3y_train = df_surv_probs_train.iloc[:,3]
suvr_prob_3y_test = df_surv_probs_test.iloc[:,3]

# extract status
# get 3-year event status based on TIMETOEVENT and MORTSTAT
status_3y_train_surv = np.where((df_train["MORTSTAT"] == 1) & (df_train["TIMETOEVENT"] <= 3), 0, 1)
status_3y_test_surv = np.where((df_test["MORTSTAT"] == 1) & (df_test["TIMETOEVENT"] <= 3), 0, 1)

# log-loss
log_loss_3y_train = log_loss(status_3y_train_surv, suvr_prob_3y_train)
log_loss_3y_test = log_loss(status_3y_test_surv, suvr_prob_3y_test)

# compute Brier
bs_train_3y = brier_score_loss(status_3y_train_surv, suvr_prob_3y_train)
bs_test_3y = brier_score_loss(status_3y_test_surv, suvr_prob_3y_test)

# c-index
c_is = c_index_classification(df_train["MORTSTAT"], suvr_prob_3y_train, df_train['TIMETOEVENT'])

print(f'Log Loss is: {log_loss_3y_train:.4f}')
print(f'Log Loss os: {log_loss_3y_test:.4f}')
print(f'BS is: {bs_train_3y:.4f}')
print(f'BS os: {bs_test_3y:.4f}')
print(f'C is: {c_is:.4f}')

Log Loss is: 0.1114
Log Loss os: 0.1125
BS is: 0.0297
BS os: 0.0296
C is: 0.8916


In [30]:
# RSF2B

In [31]:
rsf2b = RandomSurvivalForest(random_state=42,
                             n_estimators=150, max_leaf_nodes=50, max_features=40, max_samples=2000)

start_time = time.time()
rsf2b.fit(X_surv_train, y_surv_train)
end_time = time.time()
print(f"Time for one model: {end_time - start_time:.2f} seconds")

Time for one model: 5.57 seconds


In [32]:
# predict survival function
surv_probs_train = rsf2b.predict_survival_function(X_surv_train)
surv_probs_test = rsf2b.predict_survival_function(X_surv_test)

# all time-to-event points
time_points_train = surv_probs_train[0].x
time_points_test = surv_probs_test[0].x 
 
# convert survival probabilities to df
df_surv_probs_train = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_train],  
    columns=time_points_train)
df_surv_probs_test = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_test],  
    columns=time_points_train)

# surv probabilites after 1 and 3 years
suvr_prob_3y_train = df_surv_probs_train.iloc[:,3]
suvr_prob_3y_test = df_surv_probs_test.iloc[:,3]

# extract status
# get 3-year event status based on TIMETOEVENT and MORTSTAT
status_3y_train_surv = np.where((df_train["MORTSTAT"] == 1) & (df_train["TIMETOEVENT"] <= 3), 0, 1)
status_3y_test_surv = np.where((df_test["MORTSTAT"] == 1) & (df_test["TIMETOEVENT"] <= 3), 0, 1)

# log-loss
log_loss_3y_train = log_loss(status_3y_train_surv, suvr_prob_3y_train)
log_loss_3y_test = log_loss(status_3y_test_surv, suvr_prob_3y_test)

# compute Brier
bs_train_3y = brier_score_loss(status_3y_train_surv, suvr_prob_3y_train)
bs_test_3y = brier_score_loss(status_3y_test_surv, suvr_prob_3y_test)

# c-index
c_is = c_index_classification(df_train["MORTSTAT"], suvr_prob_3y_train, df_train['TIMETOEVENT'])

print(f'Log Loss is: {log_loss_3y_train:.4f}')
print(f'Log Loss os: {log_loss_3y_test:.4f}')
print(f'BS is: {bs_train_3y:.4f}')
print(f'BS os: {bs_test_3y:.4f}')
print(f'C is: {c_is:.4f}')

Log Loss is: 0.1116
Log Loss os: 0.1127
BS is: 0.0297
BS os: 0.0296
C is: 0.8907


In [33]:
# RSF3B

In [34]:
rsf3b = RandomSurvivalForest(random_state=42,
                             n_estimators=200, max_leaf_nodes=50, max_features=None, max_samples=2000)

start_time = time.time()
rsf3b.fit(X_surv_train, y_surv_train)
end_time = time.time()
print(f"Time for one model: {end_time - start_time:.2f} seconds")

Time for one model: 8.48 seconds


In [35]:
# predict survival function
surv_probs_train = rsf3b.predict_survival_function(X_surv_train)
surv_probs_test = rsf3b.predict_survival_function(X_surv_test)

# all time-to-event points
time_points_train = surv_probs_train[0].x
time_points_test = surv_probs_test[0].x 
 
# convert survival probabilities to df
df_surv_probs_train = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_train],  
    columns=time_points_train)
df_surv_probs_test = pd.DataFrame(
    data=[fn(time_points_train) for fn in surv_probs_test],  
    columns=time_points_train)

# surv probabilites after 1 and 3 years
suvr_prob_3y_train = df_surv_probs_train.iloc[:,3]
suvr_prob_3y_test = df_surv_probs_test.iloc[:,3]

# extract status
# get 3-year event status based on TIMETOEVENT and MORTSTAT
status_3y_train_surv = np.where((df_train["MORTSTAT"] == 1) & (df_train["TIMETOEVENT"] <= 3), 0, 1)
status_3y_test_surv = np.where((df_test["MORTSTAT"] == 1) & (df_test["TIMETOEVENT"] <= 3), 0, 1)

# log-loss
log_loss_3y_train = log_loss(status_3y_train_surv, suvr_prob_3y_train)
log_loss_3y_test = log_loss(status_3y_test_surv, suvr_prob_3y_test)

# compute Brier
bs_train_3y = brier_score_loss(status_3y_train_surv, suvr_prob_3y_train)
bs_test_3y = brier_score_loss(status_3y_test_surv, suvr_prob_3y_test)

# c-index
c_is = c_index_classification(df_train["MORTSTAT"], suvr_prob_3y_train, df_train['TIMETOEVENT'])

print(f'Log Loss is: {log_loss_3y_train:.4f}')
print(f'Log Loss os: {log_loss_3y_test:.4f}')
print(f'BS is: {bs_train_3y:.4f}')
print(f'BS os: {bs_test_3y:.4f}')
print(f'C is: {c_is:.4f}')

Log Loss is: 0.1113
Log Loss os: 0.1122
BS is: 0.0297
BS os: 0.0296
C is: 0.8916
